## 패션 이미지 데이터셋 전처리

In [1]:
# import pandas as pd

# path = 'data/meta/'
# train = pd.read_json(path + 'train_no_dup.json')
# valid = pd.read_json(path + 'valid_no_dup.json')
# test = pd.read_json(path + 'test_no_dup.json')
# blank = pd.read_json(path + 'fill_in_blank_test.json')

## Data Load Example

In [1]:
import pandas as pd
from dataloader.sampling import DataSampler
from dataloader.multimodal_data import MultiModalData
import os

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
meta_dir = os.path.join(data_dir, 'meta')
image_dir = os.path.join(data_dir, 'images')
sampler = DataSampler(data_path = meta_dir, k=150, test_sampling_ratio=1)
concat_df, question_data = sampler.sample_data()

train_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='train')
valid_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='valid')
test_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, question = question_data, mode='test')

In [2]:
test_dataset[10].keys()

c:\ProgramData\Anaconda3\envs\jin\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


dict_keys(['texts', 'prices', 'likes', 'images', 'set_id', 'question', 'valid_idx', 'gt_idx'])

In [3]:
train_dataset[10]

{'texts': tensor([[ 0.1232,  0.3848, -0.2727,  ...,  0.3809,  0.4153,  0.0362],
         [-0.1003,  0.5273, -0.3535,  ...,  0.3945, -0.3074,  0.4644],
         [ 0.0574, -0.1785, -0.0731,  ...,  0.3362,  0.1958,  0.0703],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
        device='cuda:0', grad_fn=<CatBackward0>),
 'text_gt': tensor([ 2.9199e-01, -4.5557e-01, -3.7427e-01,  2.5244e-01, -1.6760e-01,
          2.9956e-01, -2.4353e-01, -2.8101e-01,  2.9639e-01,  2.6685e-01,
         -2.1375e-01, -5.0977e-01,  2.3926e-01, -8.7952e-02, -1.8005e-01,
          2.0483e-01, -2.8149e-01, -5.9204e-02, -5.6982e-01,  4.3579e-01,
         -5.4474e-02, -1.9897e-02,  2.4951e-01,  6.0120e-03,  2.7881e-01,
         -1.6821e-01, -3.1152e-01, -6.7688e-02, -3.0957e-01, -1.5881e-01,
         -2.0508e-01, -1.1725e-01,  1.8628e-0

## Dataloader

In [17]:
# def custom_collate_fn(batch, test=False):
#     texts = [item["texts"] for item in batch]
#     price = [item["price"] for item in batch]         
#     likes = [item["likes"] for item in batch]         
#     images = [item["images"] for item in batch]        
#     set_id = [item["set_id"] for item in batch]       
       
#     if test:
#         question = [item["question"] for item in batch] 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id,
#             "question": question,
#         }
#     else: 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id
#         }

In [2]:
from torch.utils.data import Dataset, DataLoader
from model.models import Multifusion, MultiLSTM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=True)

In [8]:
# Dataloader

for batch_idx, batch in enumerate(train_dataloader):
    print(batch.keys())
    break

dict_keys(['texts', 'text_gt', 'prices', 'likes', 'images', 'image_gt', 'set_id', 'valid_idx', 'gt_idx'])


In [6]:
train_dataloader

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
mlp = Multifusion().to(device)
lstm = MultiLSTM(input_size=512, hidden_size=512, num_layers=1, bidirectional=True).to(device)

In [5]:
import torch.nn.functional as F

optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3)

def cosine_similarity_loss(image_context, image_gt, text_context, text_gt):
    image_loss = 1 - F.cosine_similarity(image_context, image_gt, dim=-1).mean()
    text_loss = 1 - F.cosine_similarity(text_context, text_gt, dim=-1).mean()
    total_loss = (image_loss + text_loss) / 2
    return total_loss

num_epochs = 2

for epoch in range(num_epochs):
    for batch_idx, batch in enumerate(dataloader):
        image_embedding = batch['images'].to(device)
        text_embedding = batch['texts'].to(device)




        image_context, text_context = lstm(image_embedding, text_embedding, gt_idx)

        loss = cosine_similarity_loss(image_context, image_gt, text_context, text_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}], Loss: {loss.item():.4f}")

torch.save(lstm.state_dict(), "multilstm_model_epoch50.pth")
print("Model saved to multilstm_model_epoch50.pth")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [24]:
from model.models import Multifusion